In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
data2014 = pd.read_csv("Output/2014dataset.csv",low_memory=False)
data2015 = pd.read_csv("Output/2015dataset.csv",low_memory=False)
# data2016 = pd.read_csv("Output/2016dataset.csv",low_memory=False) doesn't exist yet

In [3]:
states = data2014["STATE"].drop_duplicates().as_matrix()
print(states)

['AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DC' 'DE' 'FL' 'GA' 'HI' 'IA' 'ID' 'IL'
 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO' 'MS' 'MT' 'NC' 'ND' 'NE'
 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD' 'TN' 'TX' 'UT'
 'VA' 'WA' 'WI' 'WV' 'WY']


In [4]:
# weighted rigorous statistics
# group can be i for individual or sg for small group
# absval can be 1 for yes and 0 for no

# functions:
# def getmm(group) - returns member months column
# xbarybar(dataset, column_name, state, group, absval) - returns weighted avg of a column
# sigmaxsigmay(dataset, column_name, state, group, absval) - returns weighted std of a column
# covxy(dataset, column_name1, column_name2, state, group, absval1, absval2) - returns weighted cov of two columns
# covspecial(dataset, column_name1, yi, ybar, state, group, absval1) - returns weighted cov of one column and one state measure
# r(dataset, column_name1, column_name2, state, group, absval1, absval2) - returns weighted r of two columns
# rspecial(dataset, column_name1, yi, ybar, sigmay, state, group, absval1) - returns weighted r of one column and one state measure
# rsquared(dataset, column_name1, column_name2, state, group, absval1, absval2) - returns weighted r^2 of two columns

def getmm(group):
    if group == "i":
        return "7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15"
    elif group == "sg":
        return "7.4Membermonths7HealthInsuranceSMALLGROUPTotalasof3/31/15"
    else:
        return ""

def xbarybar(dataset, column_name, state, group, absval):
    if state == "ALL":
        temp = dataset
    else:
        temp = dataset.where(dataset["STATE"] == state).dropna(how='all')
    mm = getmm(group)
    sum_mm = sum(temp[mm].convert_objects(convert_numeric=True).dropna())
    if sum_mm == 0:
        return
    numerator = 0
    denominator = 0
    for index, row in temp.iterrows():
        if (pd.notnull(row[column_name])) and (pd.notnull(row[mm])):
            ratio = row[mm] / sum_mm
            denominator = denominator + ratio
            if absval == 1:
                numerator = numerator + (abs(row[column_name]) * ratio)
            else:
                numerator = numerator + (row[column_name]  * ratio)
    return numerator / denominator

def xbarybarspecial(dataset, statedataset, column_name2, group):
    states = dataset["STATE"].drop_duplicates()
    # find the weighted average of plrs by member month
    mm = getmm(group)
    temp = dataset
    sum_mm = sum(temp[mm].convert_objects(convert_numeric=True).dropna())
    totalsum = 0
    i = 0
    for state in states:
        temp2 = statedataset.where(statedataset["State"] == state).dropna(how='all')
        if len(temp2.index) > 0:
            temp = dataset.where(dataset["STATE"] == state).dropna(how='all')
            sum_state_mm = sum(temp[mm].convert_objects(convert_numeric=True).dropna())
            if (pd.notnull(temp2[column_name2][i])):
                totalsum = totalsum + (temp2[column_name2][i] * sum_state_mm / sum_mm)
            i = i + 1
    return totalsum

def sigmaxsigmay(dataset, column_name, state, group, absval):
    if state == "ALL":
        temp = dataset
    else:
        temp = dataset.where(dataset["STATE"] == state).dropna(how='all')
    mm = getmm(group)
    sum_mm = sum(temp[mm].convert_objects(convert_numeric=True).dropna())
    if sum_mm == 0:
        return "-1"
    numerator = 0
    denominator = 0
    xbar = xbarybar(dataset, column_name, state, group, absval)
    for index, row in temp.iterrows():
        if (pd.notnull(row[column_name])) and (pd.notnull(row[mm])):
            ratio = row[mm] / sum_mm
            denominator = denominator + ratio
            if absval == 1:
                numerator = numerator + ((abs(row[column_name]) - abs(xbar))**2 * ratio)
            else:
                numerator = numerator + ((row[column_name] - xbar)**2 * ratio)
    return np.sqrt(numerator / denominator)

def sigmaxsigmayspecial(dataset, statedataset, column_name2, group):
    states = dataset["STATE"].drop_duplicates()
    # find the weighted average of plrs by member month
    mm = getmm(group)
    temp = dataset
    sum_mm = sum(temp[mm].convert_objects(convert_numeric=True).dropna())
    totalsum = 0
    i = 0
    xbar = xbarybarspecial(dataset, statedataset, column_name2, group)
    for state in states:
        temp2 = statedataset.where(statedataset["State"] == state).dropna(how='all')
        if len(temp2.index) > 0:
            temp = dataset.where(dataset["STATE"] == state).dropna(how='all')
            sum_state_mm = sum(temp[mm].convert_objects(convert_numeric=True).dropna())
            if (pd.notnull(temp2[column_name2][i])):
                totalsum = totalsum + ((temp2[column_name2][i] - xbar)**2 * sum_state_mm / sum_mm)
            i = i + 1
    return np.sqrt(totalsum)

def covxy(dataset, column_name1, column_name2, state, group, absval1, absval2):
    if state == "ALL":
        temp = dataset
    else:
        temp = dataset.where(dataset["STATE"] == state).dropna(how='all')
    mm = getmm(group)
    sum_mm = sum(temp[mm].convert_objects(convert_numeric=True).dropna())
    if sum_mm == 0:
        return -1
    numerator = 0
    denominator = 0
    xbar = xbarybar(dataset, column_name1, state, group, absval1)
    ybar = xbarybar(dataset, column_name2, state, group, absval2)
    for index, row in temp.iterrows():
        if (pd.notnull(row[column_name2])) and (pd.notnull(row[column_name1])) and (pd.notnull(row[mm])):
            xi = row[column_name1]
            yi = row[column_name2]
            ratio = row[mm] / sum_mm
            denominator = denominator + ratio
            if absval1 == 1 and absval2 == 1:
                numerator = numerator + ((abs(xi) - xbar) * (abs(yi) - ybar) * ratio)
            elif absval1 == 1:
                numerator = numerator + ((abs(xi) - xbar) * (yi - ybar) * ratio)
            elif absval2 == 1:
                numerator = numerator + ((xi - xbar) * (abs(yi) - ybar) * ratio)
            else:
                numerator = numerator + ((xi - xbar) * (yi - ybar) * ratio)
    return numerator / denominator

def covspecial(dataset, column_name1, statedataset, column_name2, group, absval1):
    temp = dataset
    mm = getmm(group)
    sum_mm = sum(temp[mm].convert_objects(convert_numeric=True).dropna())
    if sum_mm == 0:
        return -1
    ybar = xbarybarspecial(dataset, statedataset, column_name2, group)
    xbar = xbarybar(dataset, column_name1, "ALL", group, absval1)
        
    numerator = 0
    denominator = 0
    i = 0
    for state in states:
        temp3 = temp.where(temp["STATE"] == state).dropna(how = 'all')
        temp2 = statedataset.where(statedataset["State"] == state).dropna(how='all')
        if len(temp2.index) > 0:
            yi = temp2[column_name2][i]
            for index, row in temp3.iterrows():
                if (pd.notnull(row[column_name1])) and (pd.notnull(row[mm])):
                    xi = row[column_name1]
                    state = row["STATE"]
                    ratio = row[mm] / sum_mm
                    denominator = denominator + ratio
                    if absval1 == 1:
                        numerator = numerator + ((abs(xi) - xbar) * (yi - ybar) * ratio)
                    else:
                        numerator = numerator + ((xi - xbar) * (yi - ybar) * ratio)
            i = i + 1
    return numerator / denominator

def r(dataset, column_name1, column_name2, state, group, absval1, absval2):
    print(str(column_name1) + " vs " + str(column_name2) + " in " + str(state))
    cov = covxy(dataset, column_name1, column_name2, state, group, absval1, absval2)
    sigx = sigmaxsigmay(dataset, column_name1, state, group, absval1)
    sigy = sigmaxsigmay(dataset, column_name2, state, group, absval2)
    r = cov / (sigx * sigy)
    print("Covariance: " + str(cov))
    print("Std Column 1: " + str(sigx))
    print("Std Column 2: " + str(sigy))
    print("r: " + str(r) + "\n")
    return r

def rspecial(dataset, column_name1, statedataset, column_name2, group, absval1):
    cov = covspecial(dataset, column_name1, statedataset, column_name2, group, absval1)
    sigx = sigmaxsigmay(dataset, column_name1, "ALL", group, absval1)
    sigy = sigmaxsigmayspecial(dataset, statedataset, column_name2, group)
    r = cov / (sigx * sigy)
    print("Covariance: " + str(cov))
    print("Std Column 1: " + str(sigx))
    print("Std Column 2: " + str(sigy))
    print("r: " + str(r) + "\n")
    return r

def rsquared(dataset, column_name1, column_name2, state, group, absval1, absval2):
    return r(dataset, column_name1, column_name2, state, group, absval1, absval2)**2

In [5]:
def play(dataset):
    playground1 = pd.read_csv(dataset,low_memory=False)

    playground1["IndPremiumPMM"] = playground1["1.1Directpremiumwritten2HealthInsuranceINDIVIDUALTotalasof3/31/15"] / playground1["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15"]
    playground1["IndCostsPMM"] = playground1["2.16Totalincurredclaims2HealthInsuranceINDIVIDUALTotalasof3/31/15"] / playground1["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15"]
    playground1["IndRTPMM"] = playground1["HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL MARKET, INCLUDING CATASTROPHIC)"] / playground1["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15"]
    playground1["IndPremiumMinusCostsPMM"] = playground1["IndPremiumPMM"] - playground1["IndCostsPMM"]

    print("Statistics for " + dataset + "\n")
    r(playground1, "IndPremiumPMM","IndRTPMM","ALL", "i", 0, 1)
    r(playground1, "IndCostsPMM","IndRTPMM","ALL", "i", 0, 1)
    r(playground1, "IndPremiumMinusCostsPMM","IndRTPMM","ALL", "i", 0, 1)
    r(playground1, "IndPremiumPMM","IndRTPMM","CA", "i", 0, 1)
    r(playground1, "IndPremiumPMM","IndRTPMM","WY", "i", 0, 1)

In [6]:
data1 = pd.read_csv("Output/2015dataset.csv",low_memory=False)
data1["IndRTPMM"] = data1["HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL MARKET, INCLUDING CATASTROPHIC)"] / data1["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15"]
data2 = pd.read_csv("Input/June2015BYindividualnoformatting.csv",low_memory=False)
#xbarybarspecial(data1, data2, "STATE AVERAGE PLAN LIABILITY RISK SCORE", "i")
#sigmaxsigmayspecial(data1, data2, "STATE AVERAGE PLAN LIABILITY RISK SCORE", "i")
rspecial(data1, "IndRTPMM",
         data2, "STATE AVERAGE PLAN LIABILITY RISK SCORE", "i", 1)

Covariance: 0.16295520427
Std Column 1: 34.1042421947
Std Column 2: 0.167642624248
r: 0.0285020029428



0.028502002942817142

In [7]:
data2.head()

,State,STATE AVERAGE MONTHLY PREMIUMS,STATE AVERAGE PLAN LIABILITY RISK SCORE,STATE AVERAGE ALLOWABLE RATING FACTOR,STATE AVERAGE ACTUARIAL VALUE,STATE INDUCED DEMAND FACTOR,STATE BILLABLE MEMBER MONTHS
0,AK,626.670030,1.575311,1.601929,0.665172,1.020881,2.520701e+05
1,AL,344.494789,1.909181,1.570782,0.709666,1.037660,2.471190e+06
2,AR,358.229595,2.075041,1.486412,0.700076,1.030931,3.264612e+06
3,AZ,295.276671,1.583898,1.514095,0.708991,1.040687,2.640412e+06
4,CA,380.020141,1.343692,1.554204,0.700477,1.038422,2.431767e+07


In [8]:
data1 = pd.read_csv("Output/2015dataset.csv",low_memory=False)
data1["IndRTPMM"] = data1["HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL MARKET, INCLUDING CATASTROPHIC)"] / data1["7.4Membermonths2HealthInsuranceINDIVIDUALTotalasof3/31/15"]
data2 = pd.read_csv("Input/June2015BYindividualnoformatting.csv",low_memory=False)
#xbarybarspecial(data1, data2, "STATE AVERAGE PLAN LIABILITY RISK SCORE", "i")
#sigmaxsigmayspecial(data1, data2, "STATE AVERAGE PLAN LIABILITY RISK SCORE", "i")
rspecial(data1, "IndRTPMM",
         data2, "STATE AVERAGE ACTUARIAL VALUE", "i", 1)

Covariance: 0.106981754266
Std Column 1: 34.1042421947
Std Column 2: 0.0170003256546
r: 0.18452026538



0.18452026538028005

In [9]:
rspecial(data1, "IndRTPMM",
         data2, "STATE AVERAGE ALLOWABLE RATING FACTOR", "i", 1)

Covariance: -0.5256212843
Std Column 1: 34.1042421947
Std Column 2: 0.121457906536
r: -0.126893316389



-0.1268933163885832

In [11]:
rspecial(data1, "IndRTPMM",
         data2, "STATE INDUCED DEMAND FACTOR", "i", 1)

Covariance: 0.0536502058278
Std Column 1: 34.1042421947
Std Column 2: 0.0157869122327
r: 0.0996473592827



0.099647359282706044

In [12]:
rspecial(data1, "IndRTPMM",
         data2, "STATE AVERAGE PLAN LIABILITY RISK SCORE", "i", 0)
rspecial(data1, "IndRTPMM",
         data2, "STATE AVERAGE ACTUARIAL VALUE", "i", 0)
rspecial(data1, "IndRTPMM",
         data2, "STATE AVERAGE ALLOWABLE RATING FACTOR", "i", 0)
rspecial(data1, "IndRTPMM",
         data2, "STATE INDUCED DEMAND FACTOR", "i", 0)

Covariance: 0.0142035828076
Std Column 1: 44.4679128955
Std Column 2: 0.167642624248
r: 0.00190531466726

Covariance: -0.00232257954574
Std Column 1: 44.4679128955
Std Column 2: 0.0170003256546
r: -0.0030723211461

Covariance: 0.0311583305676
Std Column 1: 44.4679128955
Std Column 2: 0.121457906536
r: 0.00576901466536

Covariance: -0.00168713228652
Std Column 1: 44.4679128955
Std Column 2: 0.0157869122327
r: -0.00240328455689



-0.0024032845568933555